<a href="https://colab.research.google.com/github/EfeSenerr/llm-judge/blob/main/Summary_and_Evaluation_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post-Processing

## Nexis Uni -> JSON

sptriptf is a Python library to convert RTF (Rich Text Format) files to plain text.

In [ ]:
!pip install striprtf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
filename = "newspaper_ampel-aus_201-300"

In [ ]:
path = "/content/drive/My Drive/Aligning AI Project/Data"

In [ ]:
path_to_articles = f"{path}/{filename}.RTF"

In [ ]:
from striprtf.striprtf import rtf_to_text
# rtf = "some rtf encoded string"
with open(path_to_articles, 'rb') as file:
  rtf_string = file.read()

In [ ]:
decoded_rtf = rtf_string.decode('utf-8', errors='replace')
text = rtf_to_text(decoded_rtf)

In [ ]:
text

'\nUser Name: = \nDate and Time: = 2025-01-20\nJob Number: = 243165525\n\nDocuments (100)\nClient/Matter: -None-\nSearch Terms: Ampel-Aus\nSearch Type: NaturalAnd\nContent Type|Narrowed by|\nnews|: Zeitungen : Presseagenturen und Presseberichte : Fachpresse : Zeitschriften : Web News Timeline: 06 Nov 2024 an 06 Dez 2024 Source Type: Newspapers |\n\n1. "Ungewöhnlich starker Zulauf" Vier Parteien in Hessen verzeichnen jede Menge neue Mitglieder seit dem Bruch der Berliner Ampel\n\n2. Ein Besuch Berlins in besonderen Zeiten\n\n3. Ein Besuch Berlins in besonderen Zeiten\n\n4. Ein Besuch Berlins in besonderen Zeiten\n\n5. "Ungewöhnlich starker Zulauf" Vier Parteien in Hessen verzeichnen jede Menge neue Mitglieder seit dem Bruch der Berliner Ampel\n\n6. "Ungewöhnlich starker Zulauf" Vier Parteien in Hessen verzeichnen jede Menge neue Mitglieder seit dem Bruch der Berliner Ampel\n\n7. "Ungewöhnlich starker Zulauf" Vier Parteien in Hessen verzeichnen jede Menge neue Mitglieder seit dem Bruch d

In [ ]:
import json
import re

# Simulated RTF-to-text output
data = text

# Step 1: Split by multiple blank lines
parts = data.split("\n\n\n\n\n")

# Step 2: Separate metadata and articles
metadata = parts[0]  # The first part is the metadata
articles = parts[1:]  # The rest are individual articles

structured_articles = []

for i, raw_text in enumerate(articles, start=1):
    # 2) Remove extra blank lines within the article and split by line
    lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

    # 3) Extract title, source, date if available
    title = lines[0] if len(lines) > 0 else None
    source = lines[1] if len(lines) > 1 else None
    date = lines[2] if len(lines) > 2 else None

    # 4) The rest is body
    body_lines = lines[3:] if len(lines) > 3 else []
    body = "\n".join(body_lines)

    structured_articles.append({
        "article_number": i,
        "title": title,
        "source": source,
        "date": date,
        "body": body
    })

# 5) Save to JSON
output_file = f"{path}/{filename}.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(structured_articles, f, ensure_ascii=False, indent=4)

print(f"Articles saved to {output_file}")

Articles saved to /content/drive/My Drive/Aligning AI Project/Data/newspaper_ampel-aus_201-300.json


## Preprocessing: Cleaning & Filtering
- Clean the articles
- Filter the articles we want using simple keywords
- Filter the duplicate articles

In [ ]:
# TODO preprecossing of the articles (html parts still there in some articles)
import os
import json
import re

# Example keyword list
KEYWORDS = ["collapse", "German government", "finance minister", "coalition", "traffic light",
            "Ampel", "Koalition", "Regierung", "Ampel-Koalition", "Ampel-Aus", "Koalitionskrise", "Regierung", "FDP", "SPD", "Ausstiegsszenarien", "Ampelkoalition"]

def load_json_files(folder_path):
    """
    Reads all .json files from a folder and returns a list of loaded JSON objects with their filenames.
    """
    json_data_list = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json") and file_name != "config.json":
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                # Each file might contain a top-level list or dict. Adjust parsing if needed.
                data = json.load(f)
                json_data_list.append((file_name, data))
    return json_data_list


def remove_duplicates(articles):
    """
    Removes duplicates within a single list of articles by comparing (title, date).
    Returns a list of unique articles.
    """
    seen = set()
    unique_articles = []
    for article in articles:
        # We combine title and date as a key to detect duplicates
        title_date_key = (article.get("title", ""), article.get("date", ""))
        if title_date_key not in seen:
            seen.add(title_date_key)
            unique_articles.append(article)
    return unique_articles


def filter_by_keywords(articles, keywords):
    """
    Keeps only articles containing at least one keyword in either the title or body.
    """
    filtered = []
    for article in articles:
        title = article.get("title", "").lower()
        body = article.get("body", "").lower()
        if any(keyword.lower() in title or keyword.lower() in body for keyword in keywords):
            filtered.append(article)
    return filtered


def remove_html_tags(text):
    """
    Strips HTML tags from a string using a regex.
    """
    # This pattern will remove tags like <p>, </p>, <strong>, etc.
    clean_text = re.sub(r'<[^>]*>', '', text)
    return clean_text


def preprocess_articles(articles):
    """
    Applies preprocessing to each article (e.g., remove HTML tags, split body by "Body").
    """
    for article in articles:
        body_text = article.get("body", "")

        # Remove HTML tags
        body_text = remove_html_tags(body_text)

        # Split the body by the line "Body" and keep only the part after it
        body_lines = body_text.split("\n")
        split_index = None
        for idx, line in enumerate(body_lines):
            if line.strip() == "Body":
                split_index = idx
                break
        if split_index is not None:
            body_lines = body_lines[split_index + 1:]

        # Join the remaining body text and strip whitespace
        body_text = "\n".join(body_lines).strip()

        # Update the article body
        article["body"] = body_text

    return articles

def save_articles_to_json(articles, original_filename, output_folder):
    """
    Saves a list of articles to a JSON file with "_processed" added to the original filename.
    """
    file_name, file_extension = os.path.splitext(original_filename)
    processed_filename = f"{file_name}_processed{file_extension}"
    output_path = os.path.join(output_folder, processed_filename)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(articles, f, ensure_ascii=False, indent=4)

In [ ]:
folder_path = f"{path}/"  # change this to your folder
output_folder = f"{path}/output_cleaning/new/"  # change this to your output folder
all_loaded_data = load_json_files(folder_path)

# Process each JSON file
for original_filename, json_file_data in all_loaded_data:
    articles = json_file_data

    # 1) Remove duplicates in this file
    unique_articles = remove_duplicates(articles)

    # 2) Filter articles by keywords
    filtered_articles = filter_by_keywords(unique_articles, KEYWORDS)

    # 3) Preprocess articles (remove HTML tags, split body, etc.)
    processed_articles = preprocess_articles(filtered_articles)

    # 4) Save processed articles to a new JSON file with the original filename + "_processed"
    print("Processed file name:", original_filename)
    save_articles_to_json(processed_articles, original_filename, output_folder)

Saving file name: articles_weblinks.json
Saving file name: articles_newswires.json
Saving file name: articles_newspapers.json
Saving file name: newspaper_ampel-aus_0-200.json
Saving file name: newspaper_ampel-aus_201-300.json


You can find the processed files in our GDrive -> data/output_cleaning/

# Summary Pipeline

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
from openai import OpenAI
import json

# Load the API key
config_path = f"{path}/config.json" # PATH OF THE API KEY

with open(config_path, 'r', encoding='utf-8') as f:
  config = json.load(f)
  API_KEY = config.get("API_KEY")

# Define the prompt you want to send to the system
SYSTEM_PROMPT = "You are a helpful assistant that summarizes articles."

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=API_KEY)

def generate_response(input_text, system_prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_prompt
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": input_text
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=0.1, # CHANGE THIS PARAMETER IF NECESSARY
        max_completion_tokens=2048, # CHANGE THIS PARAMETER IF NECESSARY
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

# Creates a summary for a single JSON file
def process_json(prompt, input_file, output_file):
    # Read JSON file
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    # For each article in JSON, read "body" and generate summary
    for item in data:
        body_text = item.get("body", "")
        if body_text:
            summary = generate_response(f"{prompt} {body_text}", SYSTEM_PROMPT)
            item["summary"] = summary

    # Write updated data to new JSON file
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    # Also create an Excel file from the updated data
    df = pd.DataFrame(data)
    df.to_excel("summaries_short.xlsx", index=False)

In [ ]:
process_json("""Create a concise and accurate summary of the following article in English.""",
             f"{path}/output_cleaning/test_articles_prompt.json", f"{path}/output_summary/test_summaries_short.json")

In [ ]:
import os
def process_all_json_in_folder(prompt, input_folder, output_folder):
    """
    Processes all JSON files in the specified input folder and saves the results
    in the output folder.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate through all JSON files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith(".json"):
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, file_name)

            print(f"Processing {file_name}...")
            process_json(prompt, input_file_path, output_file_path)
            print(f"Saved processed file to {output_file_path}")

In [ ]:
# Define the folder paths
input_folder = f"{path}/output_cleaning"
output_folder = f"{path}/output_summary"

# Run the processing function
process_all_json_in_folder("Create a concise and accurate summary of the following article in English.",
                           input_folder, output_folder)

# For the german articles & summaries:
# process_all_json_in_folder("Erstelle eine prägnante und genaue Zusammenfassung des folgenden Artikels in deutscher Sprache.", input_folder, output_folder)

### Convert Json files to Excel


In [ ]:
import os
import glob
import pandas as pd

# Specify the folder containing the JSON files
folder_path = f"{path}/output_summary/0402_summaries_english"  # Change this to your target folder

# Find all JSON files in the folder
json_files = glob.glob(os.path.join(folder_path, "*.json"))

for json_file in json_files:
    try:
        # Load JSON file into a DataFrame
        df = pd.read_json(json_file)

        # Create an Excel file name with the same base name
        base_name = os.path.splitext(os.path.basename(json_file))[0]
        excel_file = os.path.join(folder_path, f"{base_name}.xlsx")

        # Write the DataFrame to an Excel file (without row indices)
        df.to_excel(excel_file, index=False)
        print(f"Converted {json_file} to {excel_file}")
    except Exception as e:
        print(f"Failed to convert {json_file}: {e}")


### Check: How many articles have been collected

In [ ]:
import os
import glob
import pandas as pd
import json

# Specify the folder path containing JSON files
folder_path = f"{path}/output_summary/0402_summaries_english"  # Change this to your target folder


# Find all JSON files in the folder
json_files = glob.glob(os.path.join(folder_path, "*.json"))

total_rows = 0

for json_file in json_files:
    try:
        with open(json_file, "r", encoding="utf-8") as f:
            content = json.load(f)

        # Determine the number of rows (records) based on the JSON structure
        if isinstance(content, list):
            # JSON is a list; each element is an Excel row
            num_rows = len(content)
        elif isinstance(content, dict):
            # If the dict has one key and its value is a list, assume that's the table
            if len(content) == 1:
                first_val = next(iter(content.values()))
                if isinstance(first_val, list):
                    num_rows = len(first_val)
                else:
                    # If not a list, count each key-value pair as one row (less common)
                    num_rows = len(content)
            else:
                # For a dict with multiple keys, you might decide to sum the lengths
                # of any list values; here we sum only lists found as values.
                num_rows = sum(len(v) for v in content.values() if isinstance(v, list()))
                # If no list is found, fallback to counting key-value pairs
                if num_rows == 0:
                    num_rows = len(content)
        else:
            num_rows = 0

        total_rows += num_rows
        print(f"{os.path.basename(json_file)}: {num_rows} rows")
    except Exception as e:
        print(f"Error reading {os.path.basename(json_file)}: {e}")

print(f"Total rows across all JSON files: {total_rows}")



articles_newspapers_processed_modified.json: 10 rows
articles_newswires_processed_modified.json: 10 rows
articles_newspapers_processed.json: 116 rows
articles_newswires_processed.json: 57 rows
Total rows across all JSON files: 193


# Evaluation Pipeline

In [ ]:
#optional if starting here
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Aligning AI Project/Data"
config_path = f"{path}/config.json" # PATH OF THE HF KEY

Mounted at /content/drive


In [ ]:
from huggingface_hub import login
import json
import transformers
import torch

with open(config_path, 'r', encoding='utf-8') as f:
  config = json.load(f)
  LLAMA3_1_token = config.get("API_LLAMA_3.1")

login(LLAMA3_1_token)
# we use llama for sentiment analysis

# Load Llama 3.1 model from Hugging Face
model_id = "meta-llama/Llama-3.1-8B-Instruct"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    temperature = 0.1, # change temperature if necessary
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
accuracy_eval_prompt = """Decide if the following summary is consistent with the corresponding article.
You find both below this instruction. Note that consistency means all information in the summary is supported by the article. answer with ANSWER:1 if the article and the summary are consistent or ANSWER:0 if any information in the summary is not supported by the article. If the texts do not mention the coalition break down, respond with WRONG_CONTENT. The only output you give should be one of the three options."""

article_eval_prompt = """Analyze the sentiment in the given text toward each of these entities: {Olaf Scholz, SPD, Robert Habeck, die Gruenen, Christian Lindner, FDP, the coalition breakdown}. Focus on the authors opinion and not on quotes.

Assign a sentiment score for each entity:
-1 = Negative, 0 = Neutral, 1 = Positive
If an entity is not mentioned or only briefly referenced, mark it as N/A.
Consider tone, language, and context when determining sentiment.
Ignore mentions that do not provide meaningful sentiment cues.
Examples for Sentiment Scoring:

Negative (-1): [The government of Germany was blown apart.]
Neutral (0): [Scholz has argued for easing spending rules to address urgent national and international challenges]
Positive (1): [Habeck geht damit auf Finanzminister Christian Lindner zu] Make sure to not judge (indirect) citations in the summary that are subjective but instead only if there is an overhang in citations with a certain narrative so that it reflects a sentiment of the author.
Use these benchmarks as guidance. Your answer needs to be of the following format, where you replace the x with the number or N/A. It is very important that you stick to this format and do not write anything else: Olaf Scholz:x, SPD:x, Robert Habeck:x, die Gruenen:x, Christian Lindner:x, FDP:x, the coalition breakdown:x"""

summary_eval_prompt = """Analyze the sentiment in the given text toward each of these entities: {Olaf Scholz, SPD, Robert Habeck, die Gruenen, Christian Lindner, FDP, the coalition breakdown}. Focus on the authors opinion and not on quotes.

Assign a sentiment score for each entity:
-1 = Negative, 0 = Neutral, 1 = Positive
If an entity is not mentioned or only briefly referenced, mark it as N/A.
Consider tone, language, and context when determining sentiment.
Ignore mentions that do not provide meaningful sentiment cues.
Examples for Sentiment Scoring:

Negative (-1): [The government of Germany was blown apart.]
Neutral (0): [Scholz has argued for easing spending rules to address urgent national and international challenges]
Positive (1): [Habeck geht damit auf Finanzminister Christian Lindner zu] Make sure to not judge (indirect) citations in the summary that are subjective but instead only if there is an overhang in citations with a certain narrative so that it reflects a sentiment of the author.
Use these benchmarks as guidance. Your answer needs to be of the following format, where you replace the x with the number or N/A. It is very important that you stick to this format and do not write anything else: Olaf Scholz:x, SPD:x, Robert Habeck:x, die Gruenen:x, Christian Lindner:x, FDP:x, the coalition breakdown:x"""

In [ ]:
# Define the evaluation function
def evaluate_text_with_llama(text, prompt):
    """
    Sends the text and prompt to Llama 3.1 for evaluation and returns the response.
    """
    messages = [
        {"role": "system", "content": "You are an expert evaluator for textual quality."},
        {"role": "user", "content": f"{prompt}\n\n{text}"}
    ]
    outputs = pipeline(
        messages,
        max_new_tokens=256,
    )
    return outputs[0]["generated_text"][-1]

In [ ]:
import os
import json
import pandas as pd
from openpyxl import load_workbook

# Function to evaluate and update JSON files
def evaluate_and_update_json_files(input_evaluation_folder, output_evaluation_folder, output_excel, start_index, end_index):
    results = []

    # Extract the folder name for sheet naming
    folder_name = os.path.basename(os.path.normpath(input_evaluation_folder))

    for file_name in os.listdir(input_evaluation_folder):
        if file_name.endswith(".json"):
            input_file_path = os.path.join(input_evaluation_folder, file_name)
            output_file_path = os.path.join(output_evaluation_folder, file_name)

            # Read the JSON file
            with open(input_file_path, "r", encoding="utf-8") as file:
                articles = json.load(file)  # Expecting a list of dictionaries (multiple articles per file)
            if end_index != 0:
                articles = articles[start_index:end_index]

            updated_articles = []

            for article in articles:
                # Extract article details
                article_text = article.get("body", "")
                summary_text = article.get("summary", "")
                article_number = article.get("article_number", "")
                title = article.get("title", "")

                if article_text and summary_text:
                    # Evaluate accuracy, article, and summary
                    accuracy_text = f"Article: {article_text} \n Summary: {summary_text}"
                    accuracy_evaluation = evaluate_text_with_llama(accuracy_text, accuracy_eval_prompt)
                    article_evaluation = evaluate_text_with_llama(article_text, article_eval_prompt)
                    summary_evaluation = evaluate_text_with_llama(summary_text, summary_eval_prompt)

                    # Add evaluations to the article
                    article["accuracy_evaluation"] = accuracy_evaluation
                    article["article_evaluation"] = article_evaluation
                    article["summary_evaluation"] = summary_evaluation

                    # Collect results for Excel
                    results.append({
                        "file_name": file_name,
                        "article_number": article_number,
                        "title": title,
                        "article": article_text,
                        "summary": summary_text,
                        "accuracy_evaluation": accuracy_evaluation,
                        "article_evaluation": article_evaluation,
                        "summary_evaluation": summary_evaluation
                    })
                    print(f"Updated article {article_number} in {file_name} with evaluations.")

                updated_articles.append(article)

            # Save updated JSON file
            with open(output_file_path, "w", encoding="utf-8") as file:
                json.dump(updated_articles, file, ensure_ascii=False, indent=4)

    # Save results to an Excel file
    df = pd.DataFrame(results)

    # Construct a unique sheet name using the folder name
    base_sheet_name = f"{folder_name}_Eval"

    try:
        with pd.ExcelWriter(output_excel, engine='openpyxl', mode='a') as writer:
            wb = writer.book
            existing_sheets = wb.sheetnames

            # Find the next available sheet number
            sheet_index = 1
            while f"{base_sheet_name}_{sheet_index}" in existing_sheets:
                sheet_index += 1

            sheet_name = f"{base_sheet_name}_{sheet_index}"

            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1)

            # Add prompts at the top row
            ws = writer.sheets[sheet_name]
            ws.cell(row=1, column=1, value=accuracy_eval_prompt)
            ws.cell(row=1, column=2, value=article_eval_prompt)
            ws.cell(row=1, column=3, value=summary_eval_prompt)

    except FileNotFoundError:
        with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
            sheet_name = f"{base_sheet_name}_1"
            df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=1)

            # Add prompts at the top row
            ws = writer.sheets[sheet_name]
            ws.cell(row=1, column=1, value=accuracy_eval_prompt)
            ws.cell(row=1, column=2, value=article_eval_prompt)
            ws.cell(row=1, column=3, value=summary_eval_prompt)

    print(f"Evaluation results saved to {output_excel} (Sheet: {sheet_name})")

In [ ]:
import os
# Define folder path to files
# f"{path}/output_summary/0402_summaries_german"
# f"{path}/output_summary/2102_summaries_german"
# f"{path}/output_summary/0402_summaries_english
input_folder = f"{path}/output_summary/0402_summaries_german"
output_folder = f"{path}/output_evaluation/output_0503"
output_excel = f"{path}/output_evaluation/evaluation_results_0503.xlsx"
start_index = 0
end_index = 0

# Run evaluation
evaluate_and_update_json_files(input_folder, output_folder, output_excel, start_index, end_index)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Updated article 1 in newspaper_Ampel_Koalition am Ende_52_processed.json with evaluations.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Updated article 2 in newspaper_Ampel_Koalition am Ende_52_processed.json with evaluations.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Updated article 496 in newspaper_D-Day_processed.json with evaluations.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Updated article 497 in newspaper_D-Day_processed.json with evaluations.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Updated article 498 in newspaper_D-Day_processed.json with evaluations.
Evaluation results saved to /content/drive/My Drive/Aligning AI Project/Data/output_evaluation/evaluation_results_0503.xlsx (Sheet: 0402_summaries_german_Eval_1)


## Aggregating results

In [ ]:
import os
# Define folder path to files
input_excel = f"{path}/output_evaluation/evaluation_results_0503.xlsx"
output_excel = f"{path}/output_evaluation/evaluation_results_aggregated_0503.xlsx"

In [ ]:
import pandas as pd
import ast
import re
from openpyxl import load_workbook


# Initialize a list to store extracted data
processed_data = []

# Define entity names for extraction
entities = ["Olaf Scholz", "SPD", "Robert Habeck", "die Gruenen", "Christian Lindner", "FDP", "the coalition breakdown"]

# Load the workbook and get sheet names
wb = load_workbook(input_excel)
sheet_names = wb.sheetnames

# Iterate through all sheets in the Excel file
for sheet_name in sheet_names:
    df = pd.read_excel(input_excel, sheet_name=sheet_name, header=None)  # Read without headers

    # Skip empty sheets
    if df.empty:
        continue

    # Assign column names manually (assuming a fixed structure)
    df.columns = ["A", "B", "C", "D", "E", "F", "G", "H"]  # Adjust if needed

    # Insert an empty row with the sheet name in the first column
    processed_data.append(["", sheet_name] + [""] * (len(entities) * 3 + 2))  # Offset to match column count

    # Iterate through each row, starting from row 3 (zero-based index 2)
    for index, row in df.iloc[2:].iterrows():
        article_file_name = row["A"]
        article_number = row["B"]
        article_name = row["C"]
        content_f = row["F"]
        content_g = row["G"]
        content_h = row["H"]

        # Initialize extracted values
        summary_consistency = None
        article_scores = {entity: None for entity in entities}
        summary_scores = {entity: None for entity in entities}
        comparison_scores = {entity: None for entity in entities}

        # Process column F for summary_consistency
        try:
            parsed_f = ast.literal_eval(str(content_f))
            if isinstance(parsed_f, dict) and "content" in parsed_f and "ANSWER" in parsed_f["content"]:
                match = re.search(r"ANSWER:(-?\d+)", parsed_f["content"])
                if match:
                    summary_consistency = int(match.group(1))
        except (ValueError, SyntaxError):
            pass  # Ignore parsing errors

        # Process column G for article scores
        try:
            parsed_g = ast.literal_eval(str(content_g))
            if isinstance(parsed_g, dict) and "content" in parsed_g:
                scores = dict(item.split(":") for item in parsed_g["content"].split(", "))
                for entity in entities:
                    if entity in scores and scores[entity] not in ["N/A", ""]:
                        article_scores[entity] = int(scores[entity])
        except (ValueError, SyntaxError):
            pass  # Ignore parsing errors

        # Process column H for summary scores
        try:
            parsed_h = ast.literal_eval(str(content_h))
            if isinstance(parsed_h, dict) and "content" in parsed_h:
                scores = dict(item.split(":") for item in parsed_h["content"].split(", "))
                for entity in entities:
                    if entity in scores and scores[entity] not in ["N/A", ""]:
                        summary_scores[entity] = int(scores[entity])
        except (ValueError, SyntaxError):
            pass  # Ignore parsing errors

        # Calculate comparison values
        for entity in entities:
            summary_value = summary_scores[entity]
            article_value = article_scores[entity]

            if summary_value is None:
                comparison_scores[entity] = 0  # If summary is empty, set 0
            elif summary_value == article_value:
                comparison_scores[entity] = 1  # If they match, set 1
            else:
                comparison_scores[entity] = -1  # If they don't match, set -1

        # Append processed data
        processed_data.append([
            index,  # Unique ID (starting from 1)
            article_file_name,
            article_number,
            article_name,
            summary_consistency,
            *[article_scores[entity] for entity in entities],
            *[summary_scores[entity] for entity in entities],
            *[comparison_scores[entity] for entity in entities],  # Add comparison scores
        ])

# Define column headers
headers = ["ID", "Article File Name", "Article_Number", "Article_Name", "Summary_Consistency"]
headers += [f"Article_{entity.replace(' ', '_')}" for entity in entities]
headers += [f"Summary_{entity.replace(' ', '_')}" for entity in entities]
headers += [f"Comparison_{entity.replace(' ', '_')}" for entity in entities]

# Create a new DataFrame
df_processed = pd.DataFrame(processed_data, columns=headers)

# Save to Excel
df_processed.to_excel(output_excel, index=False, engine="openpyxl")

print(f"Aggregated data saved to {output_excel}")


Aggregated data saved to /content/drive/My Drive/Aligning AI Project/Data/output_evaluation/evaluation_results_aggregated_0503.xlsx


In [ ]:
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
# === Apply Formatting in Excel ===
wb = load_workbook(output_excel)
ws = wb.active

# Define color fills
light_blue_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")  # Light blue
light_orange_fill = PatternFill(start_color="FFD580", end_color="FFD580", fill_type="solid")  # Light orange

# Apply light blue to first columns (A:D in Excel, 1-based indexing)
for col in range(1, 5):  # Columns 1 to 4 (ID, Article Number, Article Name, Summary Consistency)
    for row in range(1, ws.max_row + 1):
        ws.cell(row=row, column=col).fill = light_blue_fill

# Apply light orange to Comparison columns (Dynamically determined)
for col_idx, col_name in enumerate(headers, start=1):
    if col_name.startswith("Comparison_"):  # Identify comparison columns
        for row in range(1, ws.max_row + 1):
            ws.cell(row=row, column=col_idx).fill = light_orange_fill

# Save workbook with new formatting
wb.save(output_excel)
print(f"Formatted data saved to {output_excel}")


Formatted data saved to /content/drive/My Drive/Aligning AI Project/Data/output_evaluation/evaluation_results_aggregated_0503.xlsx
